In [0]:

from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")

client = Socrata("www.datos.gov.co",str(token),timeout=1000)

In [0]:
df_secop_id=spark.table("diplomado_datos.ids_contratos_procesos")
lista_entidades=df_secop_id.select("nit_de_la_entidad").distinct().collect()
len(list(map(lambda x: x.nit_de_la_entidad,lista_entidades)))

In [0]:

import time

mode_write = "overwrite"
reintentos = 5

for entidad in lista_entidades:
  intentos = 0
  while intentos < reintentos:
    try:
      query="""
      SELECT * WHERE nit_de_la_entidad = '{0}'
      """.format(entidad.nit_de_la_entidad)
      print(query)
      results_entidad=client.get(codigo_dataset,query=query)
      df_total=spark.createDataFrame(results_entidad)
      df_total.write \
        .format("delta") \
        .mode(mode_write) \
        .option("overwriteSchema", "true") \
        .saveAsTable("diplomado_datos.secop_bronze")
      mode_write = "append"
      break
    except Exception as e:
      intentos += 1
      time.sleep(10)
      print("Error: {0}".format(e))


print("Se cargaron todos los contratos")

In [0]:
df_secop_id=spark.table("diplomado_datos.ids_contratos_procesos")
df_secop_id.count()

In [0]:
# Importar las funciones necesarias de PySpark
from pyspark.sql.functions import sha2, concat_ws, col

# 1. Cargar la tabla correcta desde el catálogo a un DataFrame
# Se asume que esta es la tabla que contiene las columnas que mencionaste.
df_secop_id = spark.table("diplomado_datos.ids_contratos_procesos")

# 2. Definir la lista corregida de columnas para el identificador único
columnas_para_hash = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor",
    "estado_del_proceso"
]

In [0]:
# 3. Añadir la nueva columna 'id_unico'
# Se concatenan las columnas clave con un separador y se les aplica un hash SHA-2.
df_con_id = df_secop_id.withColumn(
    "id_unico_con_estado",
    sha2(concat_ws("||", *[col(c) for c in columnas_para_hash]), 256)
)

columnas_para_hash_se = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor"
]
df_con_id = df_con_id.withColumn(
    "id_unico_sin_estado",
    sha2(concat_ws("||", *[col(c) for c in columnas_para_hash_se]), 256)
)
df_con_id.display()

In [0]:

df_con_id.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("diplomado_datos.secop_id_bronze")